# Data Collection (Game 1)

In [10]:
import requests
import json
import pandas as pd
import numpy as np
from geopy import geocoders
from geopy.geocoders import Nominatim
from urllib.parse import urljoin
import posixpath


# NHL API:https://github.com/dword4/nhlapi
# https://www.dataquest.io/blog/python-api-tutorial/
# https://stackoverflow.com/questions/41100303/convert-api-to-pandas-dataframe

    
r = requests.get('https://statsapi.web.nhl.com/api/v1/game/2018030411/feed/live')


def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def geo(location):
    g = geocoders.Nominatim(user_agent='MSGT_Lab5', timeout=None)
    try:
        loc = g.geocode(location)
        return loc.latitude, loc.longitude
    except:
        return 0,0

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
# pandas data frame of allPlays
x = r.json()

# Allplays DataFrame
df = pd.DataFrame(x['liveData']['plays']['allPlays'])
df

# Creating variable for normalizing nested data
data = x['liveData']['plays']['allPlays']
data

# Flatten allPlays nested data
results_df = pd.json_normalize(data)
results_df

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html
# Coordinate single x,y
# df.iloc[5][2]

# Print coordinate Column w/out header
coords = df.iloc[:,2]

events = df.iloc[:,0][3:]

# Flatten Coordinates data
coords_df = pd.json_normalize(coords)
coords_df

# Flatten allPlays nested data
event_df = pd.json_normalize(events)

# event_df.description


period = df.iloc[:,1][2:]

period_df = pd.json_normalize(period)
period_df['period']



#     team = df.iloc[:,4][2:]


time = df.iloc[:,1][2:]
time

time_df = pd.json_normalize(time)
time_df['periodTime']
time_df['dateTime']


date = time_df['dateTime'][:9].str[:10]
date

gamedata_df = pd.DataFrame(x['gameData'])
game_num = gamedata_df['game'][0]

# # df.loc[['Dean', 'Aaron'], 'food']
# gamedata_df.loc[['home', 'away'],'teams']

# away = []
# home = []

# away = gamedata_df.iloc[10,[3]]
# away_df = pd.json_normalize(away)
# away_df

home_away = gamedata_df.iloc[[10,11]]
home_away

home_away_df = pd.json_normalize(home_away.teams.to_dict())


home_away_df['away.name']
home_away_df['home.name']
# home_away_df['away.shortName']
# home_away_df['home.shortName']
# home_away_df['away.triCode']
# home_away_df['home.triCode']


# # df.loc[:, ['food', 'color']]
# gamedata_df.loc[:,['home', 'away']]

#create empty array for putting event coordinate data in
latitude = []
longitude = []

#iterative loop for pushing data into arrays
for i in coords:

    if coords_df['x'] is not None:

        lat = coords_df['y']
        lon = coords_df['x']
        for i in [lat]:
            latitude.append(i)
        for i in [lon]:
            longitude.append(i)

# Creates datafrome with events that includes coordinates
coordinates_df = pd.DataFrame(list(zip(results_df['result.event'],results_df['result.description'],results_df['team.name'],results_df['team.triCode'],period_df['period'],time_df['periodTime'],time_df['dateTime'].str[:10],lat,lon)), columns = ['Event','Description','Team','Tricode','Period','Time','Date','Lat','Lon'])
# Removes rows with "NaN" stoppages and period ends (https://stackoverflow.com/questions/39339935/pandas-dropping-rows-with-missing-data-not-working-using-isnull-notnull)
coordinates_df.replace(["NaN", 'NaT'], np.nan, inplace = True)
coordinates_df = coordinates_df.dropna()

# must correct latitude to 95 or -95!
#     https://datatofish.com/compare-values-dataframes/
coordinates_df['Home_Away_Team'] = np.where(coordinates_df['Tricode'].values == home_away_df['away.triCode'].values, 'Away', 'Home')

coordinates_df


# SAVING TO FILE!
coordinates_df.to_csv('2019SCF_events.csv', header = True, mode = 'a')
#     coordinates_df.to_json(r'events.json')


# Mapbox & Plotly

In [232]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
import chart_studio.plotly as py
from plotly.offline import iplot
import os

token = os.getenv('MAPBOX_ACCESS_TOKEN')
# mapbox_access_token = open("mapbox_token.txt").read() # Create a local file with mapbox token

df = pd.read_csv('2019SCF_events.csv')


# All Events
events_lat = df.Lat
events_lon = df.Lon
locations_name = df.Description


# # Team Events
# STL = df[df['Team'].str.contains('St. Louis Blues')]
# STL_lat = df[df['Team'].str.contains('St. Louis Blues')]['Lat']
# STL_lon = df[df['Team'].str.contains('St. Louis Blues')]['Lon']
# STL_text = df[df['Team'].str.contains('St. Louis Blues')]['Description']

# BOS = df[df['Team'].str.contains('Boston Bruins')]
# BOS_lat = df[df['Team'].str.contains('Boston Bruins')]['Lat']
# BOS_lon = df[df['Team'].str.contains('Boston Bruins')]['Lon']
# BOS_text = df[df['Team'].str.contains('Boston Bruins')]['Description']

# Home Team Events
home = df[df['Home_Away_Team'].str.contains('Home')]
home_lat = df[df['Home_Away_Team'].str.contains('Home')]['Lat']
home_lon = df[df['Home_Away_Team'].str.contains('Home')]['Lon']
home_text = df[df['Home_Away_Team'].str.contains('Home')]['Description']


# Away Team Events
away = df[df['Home_Away_Team'].str.contains('Away')]
away_lat = df[df['Home_Away_Team'].str.contains('Away')]['Lat']
away_lon = df[df['Home_Away_Team'].str.contains('Away')]['Lon']
away_text = df[df['Home_Away_Team'].str.contains('Away')]['Description']


# Display Data for Hits
hits = df[df['Event'].str.contains('Hit')]
hits_lat = df[df['Event'].str.contains('Hit')]['Lat']
hits_lon = df[df['Event'].str.contains('Hit')]['Lon']
hits_text = df[df['Event'].str.contains('Hit')]['Description']


# Display Data for Faceoffs
faceoff = df[df['Event'].str.contains('Faceoff')]
faceoff_lat = df[df['Event'].str.contains('Faceoff')]['Lat']
faceoff_lon = df[df['Event'].str.contains('Faceoff')]['Lon']
faceoff_text = df[df['Event'].str.contains('Faceoff')]['Description']

# Display Data for Shots
shots = df[df['Event'].str.contains('Shot')]
shots_lat = df[df['Event'].str.contains('Shot')]['Lat']
shots_lon = df[df['Event'].str.contains('Shot')]['Lon']
shots_text = df[df['Event'].str.contains('Shot')]['Description']

# Display Data for Penalties rgb(0, 255, 0)
penalties = df[df['Event'].str.contains('Penalty')]
pen_lat = df[df['Event'].str.contains('Penalty')]['Lat']
pen_lon = df[df['Event'].str.contains('Penalty')]['Lon']
pen_text = df[df['Event'].str.contains('Penalty')]['Description']

# Display Data for Giveaways rgb(51, 0, 0)
giveaway = df[df['Event'].str.contains('Giveaway')]
give_lat = df[df['Event'].str.contains('Giveaway')]['Lat']
give_lon = df[df['Event'].str.contains('Giveaway')]['Lon']
give_text = df[df['Event'].str.contains('Giveaway')]['Description']

# Display Data for Takeaways rgb(51, 0, 0)
takeaway = df[df['Event'].str.contains('Takeaway')]
take_lat = df[df['Event'].str.contains('Takeaway')]['Lat']
take_lon = df[df['Event'].str.contains('Takeaway')]['Lon']
take_text = df[df['Event'].str.contains('Takeaway')]['Description']


fig = go.Figure()


# All Event Markers
fig.add_trace(go.Scattermapbox(
        lat = events_lat,
        lon = events_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 17,
            color = 'rgb(255, 0, 0)',
            opacity = 0.7
        ),
        text = locations_name,
        hoverinfo = 'text',
        name = "All Events",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = df,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))

# # BOS Event Markers
# fig.add_trace(go.Scattermapbox(
#         lat=BOS_lat,
#         lon=BOS_lon,
#         mode='markers',
#         marker=go.scattermapbox.Marker(
#             size=20,
#             color='rgb(255, 0, 0)',
#             opacity=0.7
#         ),
#         text=STL_text,
#         hoverinfo='text',
#         name="BOS"
#     ))

# # STL Event Markers
# fig.add_trace(go.Scattermapbox(
#         lat=STL_lat,
#         lon=STL_lon,
#         mode='markers',
#         marker=go.scattermapbox.Marker(
#             size=20,
#             color='rgb(255, 0, 0)',
#             opacity=0.7
#         ),
#         text=STL_text,
#         hoverinfo='text',
#         name="STL"
#     ))

# Home Event Markers
fig.add_trace(go.Scattermapbox(
        lat = home_lat,
        lon = home_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 20,
            color = 'rgb(255, 0, 0)',
            opacity = 0.7
        ),
        text = home_text,
        hoverinfo = 'text',
        name = "Home",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = home,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))

# Away Event Markers
fig.add_trace(go.Scattermapbox(
        lat = away_lat,
        lon = away_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 20,
            color = 'rgb(255, 0, 0)',
            opacity = 0.7
        ),
        text = away_text,
        hoverinfo = 'text',
        name = "Away",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = away,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))


# Hits Event Markers
fig.add_trace(go.Scattermapbox(
        lat = hits_lat,
        lon = hits_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 10,
            color = 'rgb(255,255,0)',
            opacity=0.7
        ),
        text=hits_text,
        hoverinfo='text',
        name="Hit",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=hits,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))

# Faceoff Event Markers
fig.add_trace(go.Scattermapbox(
        lat=faceoff_lat,
        lon=faceoff_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(255,165,0)',
            opacity=0.7
        ),
        text=faceoff_text,
        hoverinfo='text',
        name="Faceoff",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=faceoff,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))

# Shots Event Markers
fig.add_trace(go.Scattermapbox(
        lat=shots_lat,
        lon=shots_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0, 0, 255)',
            opacity=0.7
        ),
        text=shots_text,
        hoverinfo='text',
        name="Shot",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=shots,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))

# Penalty Event Markers
fig.add_trace(go.Scattermapbox(
        lat=pen_lat,
        lon=pen_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0, 255, 000)',
            opacity=0.7
        ),
        text=pen_text,
        hoverinfo='text',
        name="Penalty",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=penalties,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))

# Giveaway Event Markers
fig.add_trace(go.Scattermapbox(
        lat=give_lat,
        lon=give_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(51, 000, 000)',
            opacity=0.7,
        ),
        text=give_text,
        hoverinfo='text',
        name="Giveaway",
    #         https://plotly.com/python/hover-text-and-formatting/
        customdata=giveaway,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))

# Takeaway Event Markers
fig.add_trace(go.Scattermapbox(
        lat=take_lat,
        lon=take_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(51, 000, 000)',
            opacity=0.7
        ),
        text=take_text,
        hoverinfo='text',
        name="Takeaway",
    #         https://plotly.com/python/hover-text-and-formatting/
        customdata=takeaway,
        hovertemplate =
        '<b>Period</b>: %{customdata[5]}<br>'+
        '<b>Time</b>: %{customdata[6]}<br>'+
        '<b>Event</b>: %{customdata[1]}<br>'+
        '<b>Description</b>: %{customdata[2]}',
    ))



fig.update_layout(
    width=800,
    height=500,
    margin={"r":10,"t":40,"l":10,"b":20},
    title='2019 Stanley Cup Finals',
    autosize=False,
    hovermode='closest',
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"
    ),
    showlegend=True,
    legend_title='<b>Events</b>',
    legend_orientation="v",
        legend=dict(
        x=-.25,
        y=1,
        traceorder="normal",
        font=dict(
            family="Rockwell",
            size=18,
            color="black"
        ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2
    ),
    mapbox=dict(
        accesstoken=token,
        center=dict(
            lat=0,
            lon=0
        ),
        pitch=0,
        zoom=1,
        bearing=0,
        style='mapbox://styles/isaacv/ck9n5j6ch0h6p1immcg0dc3hr'
    ),
)

# Need bounds and max zoom



fig.show()

# final = fig.show()
# print(time)